In [1]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("09"))

puzzle.header()

# Disk Fragmenter

[Open Website](https://adventofcode.com/2024/day/9)

In [2]:
# helper functions
def domain_from_input(input: InputReader) -> list:
    lines = input.as_str()

    is_data = True
    counter = 0
    disk = []
    # iterate over each digit
    for i in range(len(lines)):
        digit_count = int(lines[i])
        if is_data:
            for j in range(digit_count):
                disk.append(counter)
            counter += 1
        else:
            for j in range(digit_count):
                disk.append(-1)
        is_data = not is_data

    return disk


test_input = domain_from_input(puzzle.example(0))
print(test_input)

[0, 0, -1, -1, -1, 1, 1, 1, -1, -1, -1, 2, -1, -1, -1, 3, 3, 3, -1, 4, 4, -1, 5, 5, 5, 5, -1, 6, 6, 6, 6, -1, 7, 7, 7, -1, 8, 8, 8, 8, 9, 9]


In [3]:
def is_sorted(disk: list) -> bool:
    last_number = -1
    first_space = -1
    # find the first -1
    for j in range(len(disk)):
        if disk[j] == -1 and first_space == -1:
            first_space = j
        elif disk[j] != -1:
            last_number = j

    if first_space - 1 == last_number:
        return True
    return False


def print_disk(disk: list):
    output = []
    for i in range(len(disk)):
        if disk[i] == -1:
            output.append(".")
        else:
            output.append(str(disk[i]))
    print("".join(output))


def checksum(disk: list) -> int:
    total = 0
    for i in range(len(disk)):
        if disk[i] != -1:
            total += disk[i] * i
    return total


# test case (part 1)
def part_1(reader: InputReader, debug: bool) -> int:
    disk = domain_from_input(reader)

    # starting from the end of the disk, swap any number that is not -1 with a -1 at the start of the disk
    for i in range(len(disk) - 1, -1, -1):
        if disk[i] != -1:
            for j in range(len(disk)):
                if disk[j] == -1:
                    disk[i], disk[j] = disk[j], disk[i]
                    if debug:
                        print_disk(disk)
                    break

        if is_sorted(disk):
            break

    return checksum(disk)


result = part_1(puzzle.example(0), True)
display(result)
assert result == 1928

009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............


1928

In [4]:
# real case (part 1)
result = part_1(puzzle.input(), False)
print(result)
assert result == 6356833654075

6356833654075

In [7]:
# test case (part 2)
class Sector:
    def __init__(self, start: int, size: int, number: int):
        self.start = start
        self.size = size
        self.number = number

    def __str__(self):
        return f"sector: {self.number} (start: {self.start}, size: {self.size})"


class Space:
    def __init__(self, start: int, size: int):
        self.start = start
        self.size = size

    def __str__(self):
        return f"space: (start: {self.start}, size: {self.size})"


class Disk:
    def __init__(self, disk: list, sectors: list, spaces: list):
        self.spaces = spaces
        self.sectors = sectors
        self.disk = disk
        self.sectors.sort(key=lambda x: x.start)
        self.spaces.sort(key=lambda x: x.start)

    def sector_iterator(self):
        # return the sectors in reverse position
        for i in range(len(self.sectors) - 1, -1, -1):
            next_sector = self.sectors[i]
            if next_sector.size > 0 and next_sector.start > self.first_space().start:
                yield self.sectors[i]

    def first_space(self) -> int:
        for i in range(len(self.spaces)):
            if self.spaces[i].size > 0:
                return self.spaces[i]

    def space_iterator(self):
        # return the spaces in sorted position
        for i in range(len(self.spaces)):
            if self.spaces[i].size > 0:
                yield self.spaces[i]

    def find_space(self, sector: Sector) -> Space:
        for space in self.space_iterator():
            if space.size >= sector.size and space.start <= sector.start:
                return space

    def rebuild_spaces(self):
        self.spaces = []
        start = 0
        size = 0
        for i in range(len(self.disk)):
            if self.disk[i] == -1:
                if size == 0:
                    start = i
                size += 1
            else:
                if size > 0:
                    self.spaces.append(Space(start, size))
                    size = 0
        if size > 0:
            self.spaces.append(Space(start, size))

    def move_sector(self, sector: Sector, space: Space):
        for i in range(sector.size):
            self.disk[space.start + i] = sector.number
        for i in range(sector.size):
            self.disk[sector.start + i] = -1

        # remove sector from list
        self.sectors.remove(sector)

        # rebuild spaces
        self.rebuild_spaces()

    def print(self):
        output = []
        for i in range(len(self.disk)):
            if self.disk[i] == -1:
                output.append(".")
            else:
                output.append(str(self.disk[i]))
        print("".join(output))
        print("")
        # print sectors
        for sector in self.sector_iterator():
            print(sector)

        for space in self.space_iterator():
            print(space)

    def checksum(self) -> int:
        total = 0
        for i in range(len(self.disk)):
            if self.disk[i] != -1:
                total += self.disk[i] * i
        return total


def domains_from_input(input: InputReader) -> Disk:
    lines = input.as_str()

    is_data = True
    counter = 0
    disk = []
    sectors = []
    spaces = []

    # iterate over each digit
    for i in range(len(lines)):
        digit_count = int(lines[i])
        if is_data:
            number = counter
            sectors.append(Sector(len(disk), digit_count, number))
            for j in range(digit_count):
                disk.append(counter)
            counter += 1
        else:
            spaces.append(Space(len(disk), digit_count))
            for j in range(digit_count):
                disk.append(-1)
        is_data = not is_data

    return Disk(disk, sectors, spaces)


def part_2(reader: InputReader, debug: bool) -> int:
    disk = domains_from_input(reader)

    # starting from the end of the disk, swap any number that is not -1 with a -1 at the start of the disk
    for sector in disk.sector_iterator():
        space = disk.find_space(sector)
        if space is not None:
            disk.move_sector(sector, space)
            if debug:
                print(disk.print())
                print(f"sector: {sector} -> space: {space}")
        else:
            if debug:
                print(f"sector: {sector} -> no space found")

    if debug:
        print(disk.print())

    return disk.checksum()


result = part_2(puzzle.example(0), True)
print(result)
# 00992111777.44.333....5555.6666.....8888..
assert result == 2858

0099.111...2...333.44.5555.6666.777.8888..

sector: 8 (start: 36, size: 4)
sector: 7 (start: 32, size: 3)
sector: 6 (start: 27, size: 4)
sector: 5 (start: 22, size: 4)
sector: 4 (start: 19, size: 2)
sector: 3 (start: 15, size: 3)
sector: 2 (start: 11, size: 1)
sector: 1 (start: 5, size: 3)
space: (start: 4, size: 1)
space: (start: 8, size: 3)
space: (start: 12, size: 3)
space: (start: 18, size: 1)
space: (start: 21, size: 1)
space: (start: 26, size: 1)
space: (start: 31, size: 1)
space: (start: 35, size: 1)
space: (start: 40, size: 2)
None
sector: sector: 9 (start: 40, size: 2) -> space: space: (start: 2, size: 3)
sector: sector: 8 (start: 36, size: 4) -> no space found
0099.1117772...333.44.5555.6666.....8888..

sector: 8 (start: 36, size: 4)
sector: 6 (start: 27, size: 4)
sector: 5 (start: 22, size: 4)
sector: 4 (start: 19, size: 2)
sector: 3 (start: 15, size: 3)
sector: 2 (start: 11, size: 1)
sector: 1 (start: 5, size: 3)
space: (start: 4, size: 1)
space: (start: 12, size: 3)
space:

In [8]:
# real case (part 2)
result = part_2(puzzle.input(), False)
print(result)
assert result == 6389911791746

6389911791746


In [9]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="Bonus points if you make a cool animation of this process.">Compact the amphipod's hard drive</span> (Bonus points if you make a cool animation of this process.)